In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"epicalyx","key":"d2e9f2d46c1176b6b93c1c5b50dc24d7"}'}

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d smohapat/sg-complaint

 37% 9.00M/24.4M [00:00<00:00, 21.8MB/s]
100% 24.4M/24.4M [00:00<00:00, 49.8MB/s]


In [6]:
import pandas as pd
from zipfile import ZipFile
zip_file = ZipFile('/content/sg-complaint.zip')
fields= ['Complaint-Status','Consumer-complaint-summary'] 
data=pd.read_csv(zip_file.open('train.csv'),usecols=fields)
data.head()

,Complaint-Status,Consumer-complaint-summary
0,Closed with explanation,"Seterus, Inc a déposé un faux rapport auprès d..."
1,Closed with non-monetary relief,XX / XX / XXXX La requête en faillite n ° XXXX...
2,Closed with explanation,"El XXXX / XXXX / 15, estaba preparando el vuel..."
3,Closed with explanation,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
4,Closed with explanation,J'ai obtenu un compte de crédit de soins pour ...


In [9]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import json
import pickle
import urllib

from sklearn.preprocessing import LabelBinarizer

print(tf.__version__)

1.12.0


In [10]:
data = data[pd.notnull(data['Consumer-complaint-summary'])]
data.head()

,Complaint-Status,Consumer-complaint-summary
3,Closed with explanation,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
5,Closed with explanation,The owner of my original mortgage filed for ba...
10,Closed with explanation,"My mortgage provider, United Wholesale Mortgag..."
11,Closed with explanation,My boyfriend and I bought a XXXX sofa and we h...
13,Closed with explanation,Both accounts below are settled and closed. Yo...


In [11]:
pd.set_option('max_colwidth', 1000)
data['Consumer-complaint-summary'] = data['Consumer-complaint-summary'].str.lower()
data.head()

,Complaint-Status,Consumer-complaint-summary
3,Closed with explanation,"the loan was paid in xxxx xxxx. in xxxx, 4 years after i moved from va (i moved in xxxx xx / xx / xxxx-6 months after paying the debt), a representative xxxx xxxx xxxx contacted me suddenly, indicating that my debt had not been paid and that i now owe them {$ 1100.00}. i explained that the debt was paid and, because so many years passed, i destroyed documents that i did not need to keep since the account was paid. in fact, if he owed the loan, xxxx xxxx xxxx had the right to cash the check in his possession, which he did not have because he had received it when the account was paid. that was something i asked them to provide me and they could not provide anything. despite my letter to them and the conversation on the phone, i was continually harassed and nothing was sent to me. then, in xxxx they contacted me through a collection letter for xxxx xxxx xxxx trying again to collect this debt. now there are two additional years, a total of 6 years after i paid the debt that xxxx xxxx x..."
5,Closed with explanation,"the owner of my original mortgage filed for bankruptcy. later, chase said he should pay them because they were the new owner of the debt. i went to the court and the property was not assigned to chase. my name and the original mortgage company were in writing. i contacted chase about it and they sent me a letter acknowledging that they did not have the power of attorney of my house but that they were trying to execute it. i presented the working document of the court xxxx xxxx. meanwhile, i was denied at least 4 times by a modification. now, chase has continuously filed a foreclosure on my house. they want me to modify my loan with them, but i declared bankruptcy and chase, by the way, did not reaffirm my debt and then tried to foreclose because they did not reaffirm the debt. i went back to court and now chase assigned an assignment to the deed, after they applied for a foreclosure. in ga, assignments must be made before foreclosure is filed. i retained a lawyer. he found out the ..."
10,Closed with explanation,"my mortgage provider, united wholesale mortgage, is holding me too much in my guarantee account. i have called them and i have written complaints and they have ignored me every time and / or they have turned me around. by telephone, they have said that they will solve the problem and i return the call and i receive the answer at all times. meanwhile, i am forced to pay money in a escrow account that has a lot more money than necessary to pay the required expenses. my next step is to file a class action lawsuit for respa infractions: payments from your clients' guarantee account should be an easy mathematical equation and, apparently, these guys can not do it right!"
11,Closed with explanation,"my boyfriend and i bought a xxxx sofa and we have a year to pay it without interest. before making the purchase, i asked him if he could establish automatic monthly payments and they told me he could. i just received the invoice (which is through td bank) and i connected online to set up the monthly payments and your system is designed so that it only allows me to make monthly payments for the minimum payment amount to expire, which does not will make pay. the sofa on time and then we would pay interest in an amount almost equal to the cost of the sofa itself. in addition, the loan is configured as a credit card so there are late fees if you do not pay monthly. i called the customer service, assuming there was a technical failure that would not allow me to enter a larger amount, and the associate told me that this is how the system is designed. she said that i can set recurring payments for the minimum amount and then call each month if i want to pay more than that. they promise an..."
13,Closed with explanation,"both accounts below are settled and closed. you must have a balance of zero on the credit report along with an amount due zero. please inve

In [12]:
import re
data['Consumer-complaint-summary'] = data['Consumer-complaint-summary'].str.replace('x', '')
data['Consumer-complaint-summary'] = data['Consumer-complaint-summary'].str.replace('{', '')
data['Consumer-complaint-summary'] = data['Consumer-complaint-summary'].str.replace('}', '')
data['Consumer-complaint-summary'] = data['Consumer-complaint-summary'].str.replace('/', '')
data.head()

,Complaint-Status,Consumer-complaint-summary
3,Closed with explanation,"the loan was paid in . in , 4 years after i moved from va (i moved in -6 months after paying the debt), a representative contacted me suddenly, indicating that my debt had not been paid and that i now owe them $ 1100.00. i eplained that the debt was paid and, because so many years passed, i destroyed documents that i did not need to keep since the account was paid. in fact, if he owed the loan, had the right to cash the check in his possession, which he did not have because he had received it when the account was paid. that was something i asked them to provide me and they could not provide anything. despite my letter to them and the conversation on the phone, i was continually harassed and nothing was sent to me. then, in they contacted me through a collection letter for trying again to collect this debt. now there are two additional years, a total of 6 years after i paid the debt that tries to collect for a debt that was paid in full. now , i am receiving a let..."
5,Closed with explanation,"the owner of my original mortgage filed for bankruptcy. later, chase said he should pay them because they were the new owner of the debt. i went to the court and the property was not assigned to chase. my name and the original mortgage company were in writing. i contacted chase about it and they sent me a letter acknowledging that they did not have the power of attorney of my house but that they were trying to eecute it. i presented the working document of the court . meanwhile, i was denied at least 4 times by a modification. now, chase has continuously filed a foreclosure on my house. they want me to modify my loan with them, but i declared bankruptcy and chase, by the way, did not reaffirm my debt and then tried to foreclose because they did not reaffirm the debt. i went back to court and now chase assigned an assignment to the deed, after they applied for a foreclosure. in ga, assignments must be made before foreclosure is filed. i retained a lawyer. he found out the names sig..."
10,Closed with explanation,"my mortgage provider, united wholesale mortgage, is holding me too much in my guarantee account. i have called them and i have written complaints and they have ignored me every time and or they have turned me around. by telephone, they have said that they will solve the problem and i return the call and i receive the answer at all times. meanwhile, i am forced to pay money in a escrow account that has a lot more money than necessary to pay the required epenses. my net step is to file a class action lawsuit for respa infractions: payments from your clients' guarantee account should be an easy mathematical equation and, apparently, these guys can not do it right!"
11,Closed with explanation,"my boyfriend and i bought a sofa and we have a year to pay it without interest. before making the purchase, i asked him if he could establish automatic monthly payments and they told me he could. i just received the invoice (which is through td bank) and i connected online to set up the monthly payments and your system is designed so that it only allows me to make monthly payments for the minimum payment amount to epire, which does not will make pay. the sofa on time and then we would pay interest in an amount almost equal to the cost of the sofa itself. in addition, the loan is configured as a credit card so there are late fees if you do not pay monthly. i called the customer service, assuming there was a technical failure that would not allow me to enter a larger amount, and the associate told me that this is how the system is designed. she said that i can set recurring payments for the minimum amount and then call each month if i want to pay more than that. they promise an inte..."
13,Closed with explanation,"both accounts below are settled and closed. you must have a balance of zero on the credit report along with an amount due zero. please investigate and update the cr

In [13]:
data['Complaint-Status'].unique()

array(['Closed with explanation', 'Closed', 'Closed with monetary relief',
       'Untimely response', 'Closed with non-monetary relief'],
      dtype=object)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27772 entries, 3 to 43265
Data columns (total 2 columns):
Complaint-Status              27772 non-null object
Consumer-complaint-summary    27772 non-null object
dtypes: object(2)
memory usage: 650.9+ KB


In [15]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27772 entries, 3 to 43265
Data columns (total 2 columns):
Complaint-Status              27772 non-null object
Consumer-complaint-summary    27772 non-null object
dtypes: object(2)
memory usage: 650.9+ KB


In [0]:
data_comp=data[['Consumer-complaint-summary']]
data_prod=data[['Complaint-Status']]

In [0]:
train_size = int(len(data_comp) * .999)

train_descriptions = data_comp[:train_size].astype('str')
train_prod = data_prod[:train_size]

test_descriptions = data_comp[train_size:].astype('str')
test_prod =data_prod[train_size:]

In [18]:
print(train_descriptions.shape)
print(test_descriptions.shape)

(27744, 1)
(28, 1)


In [0]:
train_size = int(len(train_descriptions) * .8)

train_desc = train_descriptions[:train_size]
train_pr = train_prod[:train_size]

val_desc = train_descriptions[train_size:]
val_pr =train_prod[train_size:]

In [20]:
print(train_desc.shape)
print(val_desc.shape)

(22195, 1)
(5549, 1)


In [21]:
print(train_pr.shape)
print(val_pr.shape)

(22195, 1)
(5549, 1)


In [22]:
print(train_desc.info())
print(val_desc.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22195 entries, 3 to 34665
Data columns (total 1 columns):
Consumer-complaint-summary    22195 non-null object
dtypes: object(1)
memory usage: 346.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5549 entries, 34667 to 43222
Data columns (total 1 columns):
Consumer-complaint-summary    5549 non-null object
dtypes: object(1)
memory usage: 86.7+ KB
None


In [23]:
from sklearn import preprocessing
encoder = preprocessing.LabelBinarizer()
encoder.fit_transform(train_pr)
train_encoded = encoder.transform(train_pr)
val_encoded = encoder.transform(val_pr)
num_classes = len(encoder.classes_)

# Print all possible products and the label for the first complaint in our training dataset
print(encoder.classes_)
print(train_encoded[0])

['Closed' 'Closed with explanation' 'Closed with monetary relief'
 'Closed with non-monetary relief' 'Untimely response']
[0 1 0 0 0]


In [24]:
description_embeddings = hub.text_embedding_column("descriptions", module_spec="https://tfhub.dev/google/universal-sentence-encoder-large/3", trainable=False)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.
INFO:tensorflow:Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/3, Total size: 810.60MB
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.


In [0]:
multi_label_head = tf.contrib.estimator.multi_label_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

In [26]:
features = {
  "descriptions": np.array(train_desc).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)
train_input_fn = tf.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=100, num_epochs=10)
estimator = tf.contrib.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[description_embeddings])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp67y8fw_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd7c9377160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [27]:
%%timeit
estimator.train(input_fn=train_input_fn)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp67y8fw_3/model.ckpt.
INFO:tensorflow:loss = 0.6937513, step = 0
INFO:tensorflow:global_step/sec: 2.81926
INFO:tensorflow:loss = 0.24595267, step = 100 (35.476 sec)
INFO:tensorflow:global_step/sec: 2.85879
INFO:tensorflow:loss = 0.2204722, step = 200 (34.981 sec)
INFO:tensorflow:glo

In [28]:
# Define our eval input_fn and run eval
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(val_desc).astype(np.str)}, val_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-23-19:21:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp67y8fw_3/model.ckpt-8880
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-23-19:21:57
INFO:tensorflow:Saving dict for global step 8880: auc = 0.9430823, auc_precision_recall = 0.80777484, average_loss = 0.22692251, global_step = 8880, loss = 0.22660342
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8880: /tmp/tmp67y8fw_3/model.ckpt-8880


{'auc': 0.9430823,
 'auc_precision_recall': 0.80777484,
 'average_loss': 0.22692251,
 'global_step': 8880,
 'loss': 0.22660342}

In [0]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(test_descriptions).astype(np.str)}, shuffle=False)

results = estimator.predict(predict_input_fn)

In [30]:
# Display predictions
for product in results:
  top = product['probabilities'].argsort()[-1:]
  for prod in top:
    text_prod = encoder.classes_[prod]
    print(text_prod + ': ' + str(round(product['probabilities'][prod] * 100, 2)) + '%')
  print('')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp67y8fw_3/model.ckpt-8880
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Closed with explanation: 85.43%

Closed with explanation: 75.39%

Closed with explanation: 72.09%

Closed with explanation: 72.22%

Closed with explanation: 73.12%

Closed with explanation: 79.31%

Closed with explanation: 76.22%

Closed with explanation: 83.41%

Closed with explanation: 69.86%

Closed with explanation: 88.72%

Closed with explanation: 82.96%

Closed with explanation: 73.52%

Closed with explanation: 85.8%

Closed with explanation: 88.37%

Closed with explanation: 80.32%

Closed with explanation: 80.95%

Closed with explanation: 88

In [31]:
test_prod

,Complaint-Status
43223,Closed with explanation
43224,Closed with explanation
43225,Closed with non-monetary relief
43227,Closed
43232,Closed with explanation
43233,Closed with explanation
43234,Closed with explanation
43235,Closed with explanation
43236,Closed with monetary relief
43237,Closed with explanation


In [7]:
!pip install langdetect

    100% |████████████████████████████████| 1.0MB 19.9MB/s 
  Running setup.py bdist_wheel for langdetect ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [8]:
from langdetect import detect


count=0
ls=[]
for i in range(0,43266):
    
    if detect(str(data["Consumer-complaint-summary"][i]))=='en':
        count+=1
        ls.append(i)
data = data.iloc[ls]        
print(count)


27772
